In [ ]:
import os

def select_images_from_folder(folder_path):
    # Get all the files in the folder
    files = os.listdir(folder_path)
    
    # Filter out non-image files (assuming .jpg and .png for simplicity)
    image_files = [file for file in files if file.endswith(('.jpg', '.jpeg', '.png'))]
    
    # Sort the image files by name
    image_files.sort()
    
    # Select images at positions 1, 11, 21, 31, etc.
    selected_images = []
    for i in range(0, len(image_files), 10):
        if i < len(image_files):
            selected_images.append(image_files[i])
    
    # Print and return the paths to the selected images
    selected_image_paths = [os.path.join(folder_path, image) for image in selected_images]
    for image_path in selected_image_paths:
        print(f'Selected image: {image_path}')
    
    return selected_image_paths

# Specify the path to the folder containing the images
folder_path = 'Images'
# Select the images
selected_image_paths = select_images_from_folder(folder_path)


In [12]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Button
%matplotlib tk

class InteractiveCurvatureCalculator:
    def __init__(self, image_paths, conversion_factor):
        self.image_paths = image_paths
        self.conversion_factor = conversion_factor  # μm/pixel
        self.current_image_index = 0
        self.points = []
        self.circles = []
        self.curvatures = []
        self.all_image_curvatures = []  # Nested list to store curvatures for all images
        self.fig, self.ax = plt.subplots(figsize=(10, 8))
        self.setup_plot()

    def setup_plot(self):
        self.ax.clear()
        self.img = plt.imread(self.image_paths[self.current_image_index])
        self.ax.imshow(self.img)
        self.ax.set_title(f'Image {self.current_image_index + 1}/{len(self.image_paths)}')
        self.fig.canvas.mpl_connect('button_press_event', self.on_click)
        self.fig.canvas.mpl_connect('key_press_event', self.on_key)

    def on_click(self, event):
        if event.inaxes != self.ax:
            return
        self.points.append((event.xdata, event.ydata))
        self.ax.plot(event.xdata, event.ydata, 'ro')
        if len(self.points) % 3 == 0:
            self.calculate_curvature()
        self.fig.canvas.draw()

    def calculate_curvature(self):
        p1, p2, p3 = self.points[-3:]
        x1, y1 = p1
        x2, y2 = p2
        x3, y3 = p3

        # Calculate the center and radius of the circle
        ma = (y2 - y1) / (x2 - x1)
        mb = (y3 - y2) / (x3 - x2)
        center_x = (ma * mb * (y1 - y3) + mb * (x1 + x2) - ma * (x2 + x3)) / (2 * (mb - ma))
        center_y = -1 / ma * (center_x - (x1 + x2) / 2) + (y1 + y2) / 2
        radius_pixels = np.sqrt((center_x - x1)**2 + (center_y - y1)**2)

        # Calculate radius in μm
        radius_um = radius_pixels * self.conversion_factor

        # Calculate curvature (1/radius) in 1/μm
        curvature = 1 / radius_um

        self.circles.append((center_x, center_y, radius_pixels))
        self.curvatures.append(curvature)

        circle = plt.Circle((center_x, center_y), radius_pixels, fill=False, color='g')
        self.ax.add_artist(circle)
        self.ax.text(center_x, center_y, f'Curvature: {curvature:.4f} 1/μm', color='g')

    def on_key(self, event):
        if event.key == 'backspace':
            if self.points:
                self.points.pop()
                if len(self.circles) > 0 and len(self.points) % 3 == 2:
                    self.circles.pop()
                    self.curvatures.pop()
                self.redraw()
        elif event.key == 'enter':
            self.all_image_curvatures.append(self.curvatures.copy())
            if self.current_image_index < len(self.image_paths) - 1:
                self.current_image_index += 1
                self.points = []
                self.circles = []
                self.curvatures = []
                self.setup_plot()
                self.fig.canvas.draw()  # Force a redraw of the figure
            else:
                plt.close(self.fig)  # Close the figure when all images are processed

    def redraw(self):
        self.ax.clear()
        self.ax.imshow(self.img)
        for point in self.points:
            self.ax.plot(point[0], point[1], 'ro')
        for circle, curvature in zip(self.circles, self.curvatures):
            c = plt.Circle(circle[:2], circle[2], fill=False, color='g')
            self.ax.add_artist(c)
            self.ax.text(circle[0], circle[1], f'Curvature: {curvature:.4f} 1/μm', color='g')
        self.fig.canvas.draw()

    def run(self):
        plt.show()

    def get_all_curvatures(self):
        return self.all_image_curvatures

# Use the conversion factor we found
conversion_factor = 0.9588011899543337  # μm/pixel

# Assume we have the selected_image_paths from the previous code
calculator = InteractiveCurvatureCalculator(selected_image_paths, conversion_factor)
calculator.run()

# After the interactive session is complete, you can access all curvatures:
all_curvatures = calculator.get_all_curvatures()

# Print curvatures for each image
for i, image_curvatures in enumerate(all_curvatures):
    print(f"Image {i+1} curvatures (1/μm): {image_curvatures}")

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats

def analyze_and_save_curvatures(all_curvatures, file_path):
    # Convert time to seconds (100s per frame)
    time_points = np.arange(len(all_curvatures)) * 100

    # Calculate average curvature and ROC for each image
    avg_curvatures = [np.mean(curvs) if curvs else np.nan for curvs in all_curvatures]
    avg_roc = [1 / np.mean(curvs) if curvs else np.nan for curvs in all_curvatures]
    
    # Calculate standard deviation of curvatures and ROC for each image
    std_curvatures = [np.std(curvs) if curvs else np.nan for curvs in all_curvatures]
    std_roc = [np.std(1 / np.array(curvs)) if curvs else np.nan for curvs in all_curvatures]

    # Create a DataFrame for time series data
    df_time_series = pd.DataFrame({
        'Time (s)': time_points,
        'Avg Curvature (1/μm)': avg_curvatures,
        'Std Curvature (1/μm)': std_curvatures,
        'Avg ROC (μm)': avg_roc,
        'Std ROC (μm)': std_roc
    })

    # Create a DataFrame for all individual curvature measurements
    all_curvatures_flat = [curvature for image_curvatures in all_curvatures for curvature in image_curvatures]
    all_roc_flat = [1/curvature for curvature in all_curvatures_flat]
    frame_numbers = [i for i, curvs in enumerate(all_curvatures) for _ in curvs]
    df_all_measurements = pd.DataFrame({
        'Frame': frame_numbers,
        'Time (s)': [fn * 100 for fn in frame_numbers],
        'Curvature (1/μm)': all_curvatures_flat,
        'ROC (μm)': all_roc_flat
    })

    # Calculate overall statistics
    overall_stats = {
        'Statistic': ['Mean', 'Median', 'Standard Deviation'],
        'Curvature (1/μm)': [
            np.mean(all_curvatures_flat),
            np.median(all_curvatures_flat),
            np.std(all_curvatures_flat)
        ],
        'ROC (μm)': [
            np.mean(all_roc_flat),
            np.median(all_roc_flat),
            np.std(all_roc_flat)
        ]
    }
    df_overall_stats = pd.DataFrame(overall_stats)

    # Trend analysis
    slope, intercept, r_value, p_value, std_err = stats.linregress(time_points, avg_curvatures)
    trend_analysis = {
        'Metric': ['Slope', 'Intercept', 'R-squared', 'P-value'],
        'Value': [slope, intercept, r_value**2, p_value],
        'Unit': ['(1/μm)/s', '1/μm', '', '']
    }
    df_trend_analysis = pd.DataFrame(trend_analysis)

    # Identify frames with extreme curvatures
    threshold = np.mean(all_curvatures_flat) + 2 * np.std(all_curvatures_flat)
    extreme_frames = [i for i, curvs in enumerate(all_curvatures) if any(curv > threshold for curv in curvs)]
    df_extreme_frames = pd.DataFrame({'Frames with extreme curvatures': extreme_frames})

    # Calculate rate of change
    rate_of_change = np.diff(avg_curvatures) / 100  # per second
    df_rate_of_change = pd.DataFrame({
        'Metric': ['Average rate of change of curvature'],
        'Value': [np.mean(rate_of_change)],
        'Unit': ['(1/μm)/s']
    })

    # Save all DataFrames to an Excel file
    with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
        df_time_series.to_excel(writer, sheet_name='Time Series Data', index=False)
        df_all_measurements.to_excel(writer, sheet_name='All Measurements', index=False)
        df_overall_stats.to_excel(writer, sheet_name='Overall Statistics', index=False)

    print(f"Analysis results saved to {file_path}")


file_path = 'curvature_analysis_results.xlsx'
analyze_and_save_curvatures(all_curvatures, file_path)